#### LangChain

从本质上分析，LangChain还是采用从大模型自身出发的策略，通过开发人员在实践过程中对打模型能力的深入理解及其在不同场景下的涌现能力，使用模块化的方式进行高级抽象，设计出统一接口以适配各种大模型。到目前为止，LangChain抽象出最终的核心模块如下：

1. Model I/O：标准化各个大模型的输入和输出，包含输入模板，模型本身和格式化输出；
2. Retrieval：检索外部数据，然后在执行生成步骤时将其传递到LLM，包括文档加载、切割、Embedding等；
3. Chains：链条，LangChain框架中最重要的模块，链接多个模块协同构建应用，是实际运作很多功能的高级抽象；
4. Memory：记忆模块。以各种方式构建历史信息，维护有关实体及其关系的信息；
5. Agents：目前最热门的Agents开发事件，未来能够真正实现通用人工智能的落地方案；
6. Callbacks：回调系统，允许连接到LLM应用程序的各个阶段。用于日志记录、监控、流传输和其他任务

### 1. 在LangChain中使用在线智谱大模型

In [6]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from dotenv import load_dotenv
from rich import print
import os
load_dotenv()
api_key = os.getenv("ZHIPU_API_KEY")

print(api_key)

c65b3dee6b47dc164cc804c4f9426f37.bKUcNHj2Tarnkdid

In [7]:
# 初始化智谱AI聊天模型
chat = ChatZhipuAI(
    model="glm-4",
    api_key=api_key,
    temperature=0.7,
    )

In [8]:
# 构建消息
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="你是一名诗人"),
    HumanMessage(content="请你帮我写一首关于鸟的诗")
]

In [ ]:
response = chat.invoke(messages)
print(response.content)

### 2. 什么是ChatPromptTemplate 

ChatPromptTemplate是LangChain提供的一个提示词模板工具，专门用于构建对话类模型的输入。

这些提示模板用于格式化消息列表

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一位乐于助人的小助理"),
    ("user", "什么是{topic}")
])
prompt_template.invoke({"topic":"什么是快乐星球"})
# chat.invoke(prompt_template.invoke({"topic":"什么是快乐星球"})).content

ChatPromptValue(messages=[SystemMessage(content='你是一位乐于助人的小助理', additional_kwargs={}, response_metadata={}), HumanMessage(content='什么是什么是快乐星球', additional_kwargs={}, response_metadata={})])

### 3. 如何构建Chat Models的LLMChain链路

In [30]:
# from langchain.chains import LLMChain
# chain = LLMChain(prompt=prompt_template, llm=chat)
# chain.invoke({"topic":"什么是快乐星球"})["text"]

# 推荐使用LCEL的语法格式构建链路
chain = prompt_template | chat  
chain.invoke({"topic":"什么是快乐星球"}).content

'快乐星球是一部中国大陆的儿童科幻电视剧，首播于2004年。这部电视剧讲述了一个来自快乐星球的神秘小机器人多啦A梦，为了拯救地球，与地球上的孩子们一起经历各种冒险的故事。\n\n在剧中，快乐星球是一个充满科技和和谐氛围的星球，那里的居民拥有高度发达的科技和丰富的想象力。快乐星球的小机器人多啦A梦来到地球，帮助地球上的孩子们解决各种问题，传播正能量，同时也让小朋友们了解到科技的力量和友谊的重要性。\n\n快乐星球因其独特的科幻元素、寓教于乐的内容以及富有童趣的角色设计，受到了广大小朋友的喜爱，成为了中国儿童电视剧的经典之作。'

### 4. Callbacks

Callbacks(回调函数)是一种编程模式,Callbacks的意义在于:允许程序在某个任务完成时自动执行另一个函数,而不必阻塞等待某个长时间运行的操作完成.所以它会在处理异步操作,如网络请求文件读写或任何可能需要等待的操作时频繁被使用

对于与大模型的连续交互场景, 一般采用的都是流式输出. 但在做流式输出前, 需要的就是回调Callbacks

回调处理程序可以是sync(同步)或async(异步)的. 同步回调和异步回调是编程中常用的两种处理事件或数据的方式,它们在执行时和用途上有明显的区别:

1. 同步回调(Synchronous Callbacks): 
    - 执行时机: 同步回调实在主程序流程中直接调用和执行的. 当一个同步回调函数被触发时, 程序会立即执行这个回调函数, 并且在回调函数执行完成之前, 主程序流程会被阻塞
    - 用途: 同步回调通常用于确保某些操作必须在程序继续执行前完成. 例如, 在访问数据中的每个元素并对齐应用函数时, 你可能会使用数据的`.foreach()`方法, 这是一个同步的回调实现
2. 异步回调(Asynchronous Callbacks):
    - **执行时机**: 异步回调不会立即执行,它们通常被放置在事件队列中,等待当前执行堆栈清空后才开始执行. 这意味着程序的主流程不会等待异步回调的完成, 可以继续执行其他任务
    - **用途**: 异步回调通常用在不希望阻塞主程序流程的情况下, 例如处理I/O操作(如网络请求, 文件读写)

在并发编程的Web开发中, 交互式应用对话场景,肯定采用的是异步的回调

In [ ]:
from typing import Any, Dict, List
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate

class LoggingHandler(BaseCallbackHandler):
    def on_chat_model_start(
            self, serialized, messages, *, run_id, parent_run_id = None, tags = None, metadata = None, **kwargs
            ) -> None:
        print("Chat model started")

    def on_llm_end(
            self, response, *, run_id, parent_run_id = None, **kwargs
            ):
        print(f"Chat model ended, response: {response}")
    
    def on_chain_start(
            self, serialized, inputs, *, run_id, parent_run_id = None, tags = None, metadata = None, **kwargs
            ):
        print(f"Chain {serialized.get('name')} started")

    def on_chain_end(
            self, outputs, *, run_id, parent_run_id = None, **kwargs
            ):
        print(f"Chain ended, outputs: {outputs}")